# General

In [1]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import re
from time import sleep

In [2]:
# global variables

in_dir = "seed"
out_dir = "processed"

email_domain = "uni.edu"

# 23-24 and 24-25 School Years: Fall 23, Winter 24, ..., Summer 2025
year_to_quarters = {
    2023: [1],
    2024: [1,2,3,4],
    2025: [2,3,4]
}

# 1 = Monday, 2 = Tuesday, 3 = Wednesday, 4 = Thursday, 5 = Friday
weekdays = [1,2,3,4,5]

# 8:00-8:50, 9:00-9:50, ..., 16:00-16:50
# timeslots = [(f"{start_hour}:00",f"{start_hour}:50") for start_hour in range(8,17)]
timeslots = [8,9,10,11,12,13,14,15,16]

In [3]:
# inserts an id column (as if generated by autoincrement) with the provided name at the provided index
def insert_id_column(df,name,index):
    df.insert(0,name,1+np.arange(index,len(df)))

In [4]:
# tries urlopen up to "attempts" time, waiting "wait_time" seconds between attempts
def stubborn_urlopen(url,attempts,wait_time):
    html = None
    i = 0
    while i < attempts:
        try:
            html = urlopen(url)
            break
        except HTTPError as e:
            sleep(wait_time)
            i += 1
    return html

In [5]:
# returns k indices uniformly at random from 0 to n-1 (order is random)
def n_choose_k(n,k):
    options = np.arange(n)
    np.random.shuffle(options)
    return options[:k]

In [6]:
# sends x to the closest value in [lower_bound,upper_bound]
def restrict(x,lower_bound,upper_bound):
    return lower_bound if x<lower_bound else upper_bound if x>upper_bound else x

restrict_01 = lambda x: restrict(x,0,1)
restrict_01_vec = np.vectorize(lambda x: 0 if x<0 else 1 if x>1 else x)
restrict_04 = lambda x: restrict(x,0,4)

# Quick Restart

Load data instead of recomputing it.

In [7]:
# student_seed and professor_seed use temporary columns that have to be recomputed

college = pd.read_csv(out_dir+"/college.csv")
department = pd.read_csv(out_dir+"/department.csv")
course = pd.read_csv(out_dir+"/course.csv")
prerequisite = pd.read_csv(out_dir+"/prerequisite.csv")
building = pd.read_csv(out_dir+"/building.csv")
room = pd.read_csv(out_dir+"/room.csv")
student = pd.read_csv(out_dir+"/student.csv")
professor = pd.read_csv(out_dir+"/professor.csv")
major_minor = pd.read_csv(out_dir+"/major_minor.csv")
section = pd.read_csv(out_dir+"/section.csv")
weekly_session = pd.read_csv(out_dir+"/weekly_session.csv")
assignment = pd.read_csv(out_dir+"/assignment.csv")
enrollment = pd.read_csv(out_dir+"/enrollment.csv")
TA = pd.read_csv(out_dir+"/TA.csv")
assignment_grade = pd.read_csv(out_dir+"/assignment_grade.csv")
final_grade = pd.read_csv(out_dir+"/final_grade.csv")

# Generate Data

## College

In [96]:
college_seed = pd.read_csv(in_dir+"/college_seed.csv").sort_values(by="Col_name").reset_index(drop=True)
college_seed

Col_name
0     College of Arts & Sciences
1  College of Built Environments
2         College of Engineering

In [97]:
insert_id_column(college_seed,"Col_id",0)
college_seed

Col_id                       Col_name
0       1     College of Arts & Sciences
1       2  College of Built Environments
2       3         College of Engineering

In [98]:
college = college_seed
college.to_csv(out_dir+"/college.csv",index=False,header=True)

## Student (tbc)

In [24]:
student_seed = pd.read_csv(in_dir+"/student_seed.csv")
student_seed

Stud_fname     Stud_lname                    Stud_email
0      Alasteir     Cattellion         acattellion0@ucsd.edu
1      Northrup       Haggidon        nhaggidon1@alibaba.com
2       Rudiger  Butler-Bowdon  rbutlerbowdon2@over-blog.com
3          Mick        Kitcatt            mkitcatt3@sohu.com
4         Boone   Spottiswoode     bspottiswoode4@boston.com
...         ...            ...                           ...
2995     Rickey          Thyer           rthyerrn@sfgate.com
2996     Alejoa           Gane          aganero@e-recht24.de
2997      Bessy        Tidbold    btidboldrp@dagondesign.com
2998       Esra           Hess         ehessrq@wordpress.org
2999   Christie    Springthorp  cspringthorprr@people.com.cn

[3000 rows x 3 columns]

In [25]:
insert_id_column(student_seed,"Stud_id",0)
student_seed

Stud_id Stud_fname     Stud_lname                    Stud_email
0           1   Alasteir     Cattellion         acattellion0@ucsd.edu
1           2   Northrup       Haggidon        nhaggidon1@alibaba.com
2           3    Rudiger  Butler-Bowdon  rbutlerbowdon2@over-blog.com
3           4       Mick        Kitcatt            mkitcatt3@sohu.com
4           5      Boone   Spottiswoode     bspottiswoode4@boston.com
...       ...        ...            ...                           ...
2995     2996     Rickey          Thyer           rthyerrn@sfgate.com
2996     2997     Alejoa           Gane          aganero@e-recht24.de
2997     2998      Bessy        Tidbold    btidboldrp@dagondesign.com
2998     2999       Esra           Hess         ehessrq@wordpress.org
2999     3000   Christie    Springthorp  cspringthorprr@people.com.cn

[3000 rows x 4 columns]

In [26]:
# change the email domains to "uni.edu"
student_seed["Stud_email"] = student_seed["Stud_email"].apply(lambda email: "".join(email.partition("@")[:2])+email_domain)
student_seed

Stud_id Stud_fname     Stud_lname              Stud_email
0           1   Alasteir     Cattellion    acattellion0@uni.edu
1           2   Northrup       Haggidon      nhaggidon1@uni.edu
2           3    Rudiger  Butler-Bowdon  rbutlerbowdon2@uni.edu
3           4       Mick        Kitcatt       mkitcatt3@uni.edu
4           5      Boone   Spottiswoode  bspottiswoode4@uni.edu
...       ...        ...            ...                     ...
2995     2996     Rickey          Thyer        rthyerrn@uni.edu
2996     2997     Alejoa           Gane         aganero@uni.edu
2997     2998      Bessy        Tidbold      btidboldrp@uni.edu
2998     2999       Esra           Hess         ehessrq@uni.edu
2999     3000   Christie    Springthorp  cspringthorprr@uni.edu

[3000 rows x 4 columns]

In [28]:
np.random.seed(5071)

# create an "aptitude" column that will later be used in generating grades
student_seed["temp_aptitude"] = restrict_01_vec(np.random.normal(0.5,0.225,size=len(student_seed)))
student_seed

Stud_id Stud_fname     Stud_lname              Stud_email  temp_aptitude
0           1   Alasteir     Cattellion    acattellion0@uni.edu       0.424435
1           2   Northrup       Haggidon      nhaggidon1@uni.edu       0.447974
2           3    Rudiger  Butler-Bowdon  rbutlerbowdon2@uni.edu       0.702458
3           4       Mick        Kitcatt       mkitcatt3@uni.edu       0.383267
4           5      Boone   Spottiswoode  bspottiswoode4@uni.edu       0.413556
...       ...        ...            ...                     ...            ...
2995     2996     Rickey          Thyer        rthyerrn@uni.edu       0.000000
2996     2997     Alejoa           Gane         aganero@uni.edu       0.177728
2997     2998      Bessy        Tidbold      btidboldrp@uni.edu       0.370496
2998     2999       Esra           Hess         ehessrq@uni.edu       0.196102
2999     3000   Christie    Springthorp  cspringthorprr@uni.edu       0.348426

[3000 rows x 5 columns]

## Building

In [10]:
building_seed = pd.read_csv(in_dir+"/building_seed.csv").sort_values(by="Bld_construction_year").reset_index(drop=True)
building_seed

Bld_name Bld_code  \
0                                          Denny Hall      DEN   
1                                     Parrington Hall      PAR   
2                                         Miller Hall      MLR   
3                                        Johnson Hall      JHN   
4                                          Smith Hall      SMI   
5                                           More Hall      MOR   
6                                        Thomson Hall      THO   
7                     Mechanical Engineering Building      MEB   
8                              Lee Paul Sieg Building      SIG   
9                                           Loew Hall      LOW   
10                                          Kane Hall      KNE   
11                                        Savery Hall      SAV   
12                                    Mary Gates Hall      MGH   
13  Bill & Melinda Gates Center for Computer Scien...     CSE2   

    Bld_construction_year  
0                    1895  
1                    1902  
2                    1922  
3                    1930  
4                    1938  
5                    1946  
6                    1948  
7                    1959  
8                    1960  
9                    1965  
10                   1971  
11                   1972  
12                   2000  
13                   2019

In [11]:
insert_id_column(building_seed,"Bld_id",0)
building_seed

Bld_id                                           Bld_name Bld_code  \
0        1                                         Denny Hall      DEN   
1        2                                    Parrington Hall      PAR   
2        3                                        Miller Hall      MLR   
3        4                                       Johnson Hall      JHN   
4        5                                         Smith Hall      SMI   
5        6                                          More Hall      MOR   
6        7                                       Thomson Hall      THO   
7        8                    Mechanical Engineering Building      MEB   
8        9                             Lee Paul Sieg Building      SIG   
9       10                                          Loew Hall      LOW   
10      11                                          Kane Hall      KNE   
11      12                                        Savery Hall      SAV   
12      13                                    Mary Gates Hall      MGH   
13      14  Bill & Melinda Gates Center for Computer Scien...     CSE2   

    Bld_construction_year  
0                    1895  
1                    1902  
2                    1922  
3                    1930  
4                    1938  
5                    1946  
6                    1948  
7                    1959  
8                    1960  
9                    1965  
10                   1971  
11                   1972  
12                   2000  
13                   2019

In [12]:
building = building_seed
building.to_csv(out_dir+"/building.csv",index=False,header=True)

## Room

I wouldn't reccommend rerunning this section unless you have to.

In [ ]:
root_url = "https://www.washington.edu"

def room_capacities(bld_code):
    building_url = root_url+"/classroom/"+bld_code
    building_html = stubborn_urlopen(building_url,5,5)
    building_bs = BeautifulSoup(building_html.read(),"html.parser")

    # rooms appear as <a href="/classroom/{building_code}+{room_number}">[whitespace]{room_name}[whitespace]</a>
    rooms = building_bs.find("ul",class_="uw-mobile-menu first-level").find_all("a",href=re.compile(f"/classroom/{bld_code}\\+[0-9]*"))
    results = {}
    for room in rooms:
        room_name = room.get_text().strip()
        room_num = room_name[len(bld_code)+1:]  # room_name = {bld_code} {room_num}

        room_url = root_url+room.attrs["href"]
        room_html = stubborn_urlopen(room_url,5,5)
        room_bs = BeautifulSoup(room_html.read(),"html.parser")

        # room capacities appear like <li>[whitespace]{number} Chairs - {Tablet Armchairs,Movable,etc.}[whitespace]</li>
        room_capacity = 0
        for entry in room_bs.find_all("li",string=re.compile("[0-9]+ Chairs")):
            line = entry.get_text().strip()
            pattern = r'([0-9]+) Chairs'
            room_capacity += int(re.match(pattern,line).groups()[0])

        results[room_num] = room_capacity
    return results

In [14]:
rows = []
for bld_code in building["Bld_code"]:
    print(bld_code)
    bld_id = building.loc[building.Bld_code == bld_code]["Bld_id"].iloc[0]
    capacities = room_capacities(bld_code)
    for room_num,capacity in capacities.items():
        rows.append({"Bld_id":bld_id,"Rm_num":room_num,"Rm_capacity":capacity})

DEN
PAR
MLR
JHN
SMI
MOR
THO
MEB
SIG
LOW
KNE
SAV
MGH
CSE2


In [15]:
room = pd.DataFrame(data=rows).sort_values(by=["Bld_id","Rm_num"]).reset_index(drop=True)
room

Bld_id Rm_num  Rm_capacity
0         1    110           20
1         1    111           50
2         1    112           40
3         1    113           35
4         1    210           20
..      ...    ...          ...
155      13    295           35
156      13    389          165
157      14    G01          110
158      14    G10          105
159      14    G20          230

[160 rows x 3 columns]

In [16]:
# may need some manual modifications
room.to_csv(out_dir+"/room.csv",index=False,header=True)

In [18]:
# reload after modifications
room = pd.read_csv(out_dir+"/room.csv")

## Department

In [20]:
department_seed = pd.read_csv(in_dir+"/department_seed.csv").sort_values(by="Dept_code").reset_index(drop=True)
department_seed

Col_name  \
0  College of Built Environments   
1         College of Engineering   
2     College of Arts & Sciences   
3     College of Arts & Sciences   
4     College of Arts & Sciences   
5         College of Engineering   
6     College of Arts & Sciences   
7     College of Arts & Sciences   
8  College of Built Environments   

                                           Dept_name Dept_code  
0                                       Architecture      ARCH  
1  Paul G. Allen School of Computer Science & Eng...       CSE  
2                            Department of Economics      ECON  
3                          Department of Linguistics      LING  
4                          Department of Mathematics      MATH  
5                             Mechanical Engineering        ME  
6                           Department of Philosophy      PHIL  
7                              Department of Physics      PHYS  
8                                        Real Estate        RE

In [21]:
insert_id_column(department_seed,"Dept_id",0)
department_seed

Dept_id                       Col_name  \
0        1  College of Built Environments   
1        2         College of Engineering   
2        3     College of Arts & Sciences   
3        4     College of Arts & Sciences   
4        5     College of Arts & Sciences   
5        6         College of Engineering   
6        7     College of Arts & Sciences   
7        8     College of Arts & Sciences   
8        9  College of Built Environments   

                                           Dept_name Dept_code  
0                                       Architecture      ARCH  
1  Paul G. Allen School of Computer Science & Eng...       CSE  
2                            Department of Economics      ECON  
3                          Department of Linguistics      LING  
4                          Department of Mathematics      MATH  
5                             Mechanical Engineering        ME  
6                           Department of Philosophy      PHIL  
7                              Department of Physics      PHYS  
8                                        Real Estate        RE

In [22]:
department_seed = department_seed.merge(college_seed,on="Col_name")[["Dept_id","Col_id","Dept_name","Dept_code"]]
department_seed

Dept_id  Col_id                                          Dept_name  \
0        1       2                                       Architecture   
1        2       3  Paul G. Allen School of Computer Science & Eng...   
2        3       1                            Department of Economics   
3        4       1                          Department of Linguistics   
4        5       1                          Department of Mathematics   
5        6       3                             Mechanical Engineering   
6        7       1                           Department of Philosophy   
7        8       1                              Department of Physics   
8        9       2                                        Real Estate   

  Dept_code  
0      ARCH  
1       CSE  
2      ECON  
3      LING  
4      MATH  
5        ME  
6      PHIL  
7      PHYS  
8        RE

In [23]:
department = department_seed
department.to_csv(out_dir+"/department.csv",index=False,header=True)

## Majors/Minors

In [99]:
np.random.seed(5071)

department_ids = department["Dept_id"].values
rows = []
for stud_id in student_seed["Stud_id"]:
    num_majors = np.random.choice([0,1,2],p=[0.25,0.5,0.25])
    if num_majors>0:
        num_minors = np.random.choice([0,1],p=[0.75,0.25])
        dept_ids = np.random.choice(department_ids,size=num_majors+num_minors,replace=False)
        major_dept_ids = dept_ids[:num_majors]
        minor_dept_ids = dept_ids[num_majors:]

        for major_dept_id in major_dept_ids:
            rows.append({"Stud_id":stud_id,"Dept_id":major_dept_id,"type":"MAJOR"})
        for minor_dept_id  in minor_dept_ids:
            rows.append({"Stud_id":stud_id,"Dept_id":minor_dept_id,"type":"MINOR"})

In [100]:
major_minor = pd.DataFrame(data=rows).sort_values(by=["Stud_id","type","Dept_id"]).reset_index(drop=True)
major_minor

Stud_id  Dept_id   type
0           3        3  MAJOR
1           3        9  MINOR
2           4        8  MAJOR
3           5        4  MAJOR
4           6        6  MAJOR
...       ...      ...    ...
3459     2997        2  MINOR
3460     2998        4  MAJOR
3461     2999        2  MAJOR
3462     3000        8  MAJOR
3463     3000        3  MINOR

[3464 rows x 3 columns]

In [101]:
major_minor.to_csv(out_dir+"/major_minor.csv",index=False,header=True)

## Professor

In [31]:
professor_seed = pd.read_csv(in_dir+"/professor_seed.csv")
professor_seed

Prof_fname   Prof_lname                     Prof_email
0       Cesaro    Trevaskis  ctrevaskis0@independent.co.uk
1      Jillana        Conan              jconan1@topsy.com
2         Fran     Lambrook         flambrook2@cbsnews.com
3    Winnifred       McAvey          wmcavey3@yolasite.com
4      Idaline  Matushevitz    imatushevitz4@mediafire.com
..         ...          ...                            ...
195    Davidde  Massingberd  dmassingberd5f@guardian.co.uk
196    Ottilie         Lugg          olugg5g@people.com.cn
197      Annie         Eble          aeble5h@earthlink.net
198    Emlynne     O'Nowlan           eonowlan5i@google.es
199     Perren      Sawfoot     psawfoot5j@hugedomains.com

[200 rows x 3 columns]

In [32]:
insert_id_column(professor_seed,"Prof_id",0)
professor_seed

Prof_id Prof_fname   Prof_lname                     Prof_email
0          1     Cesaro    Trevaskis  ctrevaskis0@independent.co.uk
1          2    Jillana        Conan              jconan1@topsy.com
2          3       Fran     Lambrook         flambrook2@cbsnews.com
3          4  Winnifred       McAvey          wmcavey3@yolasite.com
4          5    Idaline  Matushevitz    imatushevitz4@mediafire.com
..       ...        ...          ...                            ...
195      196    Davidde  Massingberd  dmassingberd5f@guardian.co.uk
196      197    Ottilie         Lugg          olugg5g@people.com.cn
197      198      Annie         Eble          aeble5h@earthlink.net
198      199    Emlynne     O'Nowlan           eonowlan5i@google.es
199      200     Perren      Sawfoot     psawfoot5j@hugedomains.com

[200 rows x 4 columns]

In [33]:
# change the email domains to "uni.edu"
professor_seed["Prof_email"] = professor_seed["Prof_email"].apply(lambda email: "".join(email.partition("@")[:2])+email_domain)
professor_seed

Prof_id Prof_fname   Prof_lname              Prof_email
0          1     Cesaro    Trevaskis     ctrevaskis0@uni.edu
1          2    Jillana        Conan         jconan1@uni.edu
2          3       Fran     Lambrook      flambrook2@uni.edu
3          4  Winnifred       McAvey        wmcavey3@uni.edu
4          5    Idaline  Matushevitz   imatushevitz4@uni.edu
..       ...        ...          ...                     ...
195      196    Davidde  Massingberd  dmassingberd5f@uni.edu
196      197    Ottilie         Lugg         olugg5g@uni.edu
197      198      Annie         Eble         aeble5h@uni.edu
198      199    Emlynne     O'Nowlan      eonowlan5i@uni.edu
199      200     Perren      Sawfoot      psawfoot5j@uni.edu

[200 rows x 4 columns]

In [34]:
np.random.seed(5071)

# randomly assign each professor to a department
professor_seed.insert(1,"Dept_id",np.random.choice(department["Dept_id"].values,size=len(professor_seed)))
professor_seed

Prof_id  Dept_id Prof_fname   Prof_lname              Prof_email
0          1        5     Cesaro    Trevaskis     ctrevaskis0@uni.edu
1          2        7    Jillana        Conan         jconan1@uni.edu
2          3        7       Fran     Lambrook      flambrook2@uni.edu
3          4        8  Winnifred       McAvey        wmcavey3@uni.edu
4          5        2    Idaline  Matushevitz   imatushevitz4@uni.edu
..       ...      ...        ...          ...                     ...
195      196        7    Davidde  Massingberd  dmassingberd5f@uni.edu
196      197        4    Ottilie         Lugg         olugg5g@uni.edu
197      198        1      Annie         Eble         aeble5h@uni.edu
198      199        8    Emlynne     O'Nowlan      eonowlan5i@uni.edu
199      200        7     Perren      Sawfoot      psawfoot5j@uni.edu

[200 rows x 5 columns]

In [46]:
professor = professor_seed
professor.to_csv(out_dir+"/professor.csv",index=False,header=True)

In [36]:
np.random.seed(5071)

# create an "aptitude" column that will later be used in generating grades
professor_seed["temp_aptitude"] = restrict_01_vec(np.random.normal(0.5,0.225,size=len(professor_seed)))
professor_seed

Prof_id  Dept_id Prof_fname   Prof_lname              Prof_email  \
0          1        5     Cesaro    Trevaskis     ctrevaskis0@uni.edu   
1          2        7    Jillana        Conan         jconan1@uni.edu   
2          3        7       Fran     Lambrook      flambrook2@uni.edu   
3          4        8  Winnifred       McAvey        wmcavey3@uni.edu   
4          5        2    Idaline  Matushevitz   imatushevitz4@uni.edu   
..       ...      ...        ...          ...                     ...   
195      196        7    Davidde  Massingberd  dmassingberd5f@uni.edu   
196      197        4    Ottilie         Lugg         olugg5g@uni.edu   
197      198        1      Annie         Eble         aeble5h@uni.edu   
198      199        8    Emlynne     O'Nowlan      eonowlan5i@uni.edu   
199      200        7     Perren      Sawfoot      psawfoot5j@uni.edu   

     temp_aptitude  
0         0.424435  
1         0.447974  
2         0.702458  
3         0.383267  
4         0.413556  
..             ...  
195       0.947850  
196       0.627486  
197       0.000000  
198       0.514212  
199       0.721658  

[200 rows x 6 columns]

## Course and Prerequsite

In [37]:
# Extracts information from a header of the form {Dept_code} {Crs_num} {Crs_name} ({Crs_credits}) {Optional: Credit type}
def parse_course_header(course_header):
    pattern = r'^.*? ([0-9]+) (.*?) \((.*)\)'
    match = re.match(pattern, course_header)
    
    course_num, course_title, course_credits = match.groups()
    return int(course_num), course_title, course_credits

parse_course_header("MATH 126 Calculus with Analytic Geometry III (5) NSc")

(126, 'Calculus with Analytic Geometry III', '5')

In [ ]:
root_url = "https://www.washington.edu/students/crscat/"

course_rows = []
prerequisite_rows = []
for department_id,department_code in zip(department["Dept_id"],department["Dept_code"]):
    print(department_code)
    
    url = root_url+str.lower(department_code)+".html"
    html = stubborn_urlopen(url,5,5)
    course_cat_bs = BeautifulSoup(html.read(),"html.parser")
    courses = course_cat_bs.find_all("a",{"name":re.compile(f"{str.lower(department_code)}[0-9]*")})

    for course in courses:
        course_header = course.b.get_text()
        course_num, course_title, course_credits = parse_course_header(course_header)
        course_prerequisites = course.get_text() \
            .partition("Prerequisite: ")[-1] \
            .partition(". Offered:")[0] \
            .partition("View course details in MyPlan")[0] \
            .split(";")
        try:
            course_num = int(course_num)
            course_credits = int(course_credits)
            course_rows.append({
                "Dept_id":department_id,
                "Crs_num":course_num,
                "Crs_name":course_title,
                "Crs_credits":course_credits,
            })
            if course_prerequisites != [""]:
                for prerequisite in course_prerequisites:
                    prerequisite_rows.append({"Crs_name":f"{department_code} {course_num}","Prereq_crs_name":prerequisite})
        except ValueError:
            # skip classes where the course_num or course_credits don't resolve to integers (e.g., course_credits = "up to 20 credits")
            pass
    

ARCH
CSE
ECON
LING
MATH
ME
PHIL
PHYS
RE


In [92]:
course_seed = pd.DataFrame(data=course_rows).sort_values(by=["Dept_id","Crs_num"]).reset_index(drop=True)
course_seed

Dept_id  Crs_num                                     Crs_name  \
0          1      100           Introduction to Architecture Study   
1          1      101        Introduction to Architectural Studies   
2          1      150               Appreciation of Architecture I   
3          1      151              Appreciation of Architecture II   
4          1      200    Architectural Design and Representation I   
..       ...      ...                                          ...   
909        9      566  Real Estate Market Evolution and Disruption   
910        9      569             Real Estate Portfolio Management   
911        9      570                  Real Estate Project Finance   
912        9      590                          Real Estate Forum I   
913        9      597                    Real Estate Data Modeling   

     Crs_credits  
0              8  
1              1  
2              3  
3              3  
4              5  
..           ...  
909            3  
910            5  
911            3  
912            1  
913            4  

[914 rows x 4 columns]

In [93]:
# drop graduate level courses
course_seed = course_seed.loc[course_seed["Crs_num"]<500]
course_seed

Dept_id  Crs_num                                    Crs_name  Crs_credits
0          1      100          Introduction to Architecture Study            8
1          1      101       Introduction to Architectural Studies            1
2          1      150              Appreciation of Architecture I            3
3          1      151             Appreciation of Architecture II            3
4          1      200   Architectural Design and Representation I            5
..       ...      ...                                         ...          ...
874        9      459  Risk and Reward in Sustainable Development            3
875        9      462      Residential Real Estate Data Analytics            5
876        9      464                          Affordable Housing            4
877        9      465                    Advanced Housing Studies            4
878        9      497                   Real Estate Data Modeling            4

[513 rows x 4 columns]

In [89]:
prerequisite_seed = pd.DataFrame(data=prerequisite_rows).sort_values(by=["Crs_name","Prereq_crs_name"]).reset_index(drop=True)
prerequisite_seed

Crs_name                                    Prereq_crs_name
0    ARCH 201                                           ARCH 200
1    ARCH 300                                           ARCH 201
2    ARCH 301                                           ARCH 300
3    ARCH 302                                           ARCH 301
4    ARCH 315                             corequisite: ARCH 300.
..        ...                                                ...
717    RE 497                                        and R E 416
718    RE 497                                            R E 250
719    RE 513                                  R E 510/URBAN 552
720    RE 519  R E 506,  URBDP 520, or a basic R software course
721    RE 550                                  R E 552/URBDP 552

[722 rows x 2 columns]

In [94]:
# going to need to manually process these files
course_seed.to_csv(in_dir+"/course_seed.csv",index=False,header=True)
prerequisite_seed.to_csv(in_dir+"/prerequisite_seed.csv",index=False,header=True)

In [78]:
course_seed.describe()

Crs_id     Dept_id     Crs_num  Crs_credits
count  914.000000  914.000000  914.000000   914.000000
mean   457.500000    4.676149  446.681619     3.879650
std    263.993371    2.494713  123.118045     1.078903
min      1.000000    1.000000   98.000000    -5.000000
25%    229.250000    2.000000  407.000000     3.000000
50%    457.500000    5.000000  473.000000     4.000000
75%    685.750000    7.000000  537.000000     5.000000
max    914.000000    9.000000  602.000000     8.000000

In [79]:
course_seed.loc[course_seed["Crs_credits"]<0]

Crs_id  Dept_id  Crs_num                        Crs_name  Crs_credits
582     583        6      495              Capstone Design II           -3
796     797        8      406  Physical Science by Inquiry II           -5

In [49]:
# reload after manual processing
course_seed = pd.read_csv(in_dir+"/course_seed.csv").sort_values(by=["Dept_id","Crs_num"]).reset_index(drop=True)
course_seed

Dept_id  Crs_num                                    Crs_name  Crs_credits
0          1      100          Introduction to Architecture Study            8
1          1      101       Introduction to Architectural Studies            1
2          1      150              Appreciation of Architecture I            3
3          1      151             Appreciation of Architecture II            3
4          1      200   Architectural Design and Representation I            5
..       ...      ...                                         ...          ...
439        9      431            Real Estate Portfolio Management            4
440        9      459  Risk and Reward in Sustainable Development            3
441        9      462      Residential Real Estate Data Analytics            5
442        9      464                          Affordable Housing            4
443        9      497                   Real Estate Data Modeling            4

[444 rows x 4 columns]

In [63]:
prerequisite_seed = pd.read_csv(in_dir+"/prerequisite_seed.csv").sort_values(by=["Crs_name","Prereq_crs_name"]).reset_index(drop=True)
prerequisite_seed

Crs_name Prereq_crs_name
0    ARCH 201        ARCH 200
1    ARCH 300        ARCH 201
2    ARCH 301        ARCH 300
3    ARCH 302        ARCH 301
4    ARCH 321        ARCH 320
..        ...             ...
324    RE 462          RE 250
325    RE 464          RE 250
326    RE 497          RE 250
327    RE 497          RE 397
328    RE 497          RE 416

[329 rows x 2 columns]

In [51]:
insert_id_column(course_seed,"Crs_id",0)
course_seed

Crs_id  Dept_id  Crs_num                                    Crs_name  \
0         1        1      100          Introduction to Architecture Study   
1         2        1      101       Introduction to Architectural Studies   
2         3        1      150              Appreciation of Architecture I   
3         4        1      151             Appreciation of Architecture II   
4         5        1      200   Architectural Design and Representation I   
..      ...      ...      ...                                         ...   
439     440        9      431            Real Estate Portfolio Management   
440     441        9      459  Risk and Reward in Sustainable Development   
441     442        9      462      Residential Real Estate Data Analytics   
442     443        9      464                          Affordable Housing   
443     444        9      497                   Real Estate Data Modeling   

     Crs_credits  
0              8  
1              1  
2              3  
3              3  
4              5  
..           ...  
439            4  
440            3  
441            5  
442            4  
443            4  

[444 rows x 5 columns]

In [67]:
def find_course_id(course_name):
    pattern = r'([A-Z]+) ([0-9]+)'
    department_code, course_num = re.match(pattern,course_name).groups()
    department_id = department.loc[department["Dept_code"]==department_code]["Dept_id"].iloc[0]
    return course_seed.loc[(course_seed["Dept_id"]==department_id) & (course_seed["Crs_num"]==int(course_num))]["Crs_id"].iloc[0]

find_course_id("RE 464")

443

In [68]:
prerequisite_seed["Crs_id"] = prerequisite_seed["Crs_name"].apply(find_course_id)
prerequisite_seed["Prereq_crs_id"] = prerequisite_seed["Prereq_crs_name"].apply(find_course_id)
prerequisite_seed

Crs_name Prereq_crs_name  Crs_id  Prereq_crs_id
0    ARCH 201        ARCH 200       6              5
1    ARCH 300        ARCH 201       9              6
2    ARCH 301        ARCH 300      10              9
3    ARCH 302        ARCH 301      11             10
4    ARCH 321        ARCH 320      13             12
..        ...             ...     ...            ...
324    RE 462          RE 250     442            428
325    RE 464          RE 250     443            428
326    RE 497          RE 250     444            428
327    RE 497          RE 397     444            431
328    RE 497          RE 416     444            438

[329 rows x 4 columns]

In [73]:
course = course_seed
course.to_csv(out_dir+"/course.csv",index=False,header=True)

In [72]:
prerequisite = prerequisite_seed[["Crs_id","Prereq_crs_id"]].sort_values(by=["Crs_id","Prereq_crs_id"]).reset_index(drop=True)
prerequisite.to_csv(out_dir+"/prerequisite.csv",index=False,header=True)

## Section and Weekly Session (tbc)

In [125]:
def generate_section_capacity(course_num):
    course_level = course_num//100
    base_capacity = 100-20*course_level    # 100, 80, 60, 40, 20
    deviation = 5*np.random.randint(-2,3)  # +- 10
    return base_capacity+deviation

In [126]:
def book_section(sect_capacity,dept_id,weekdays_,rm_availabilities,prof_availabilities):
    room_options = room.loc[room["Rm_capacity"]>=sect_capacity][["Bld_id","Rm_num"]].values
    prof_options = professor.loc[professor["Dept_id"]==dept_id]["Prof_id"].values
    np.random.shuffle(room_options)
    np.random.shuffle(prof_options)

    for bld_id,rm_num in room_options:
        # find any timeslots where the room is available on all weekdays_
        rm_available_ts_by_wd = rm_availabilities[(bld_id,rm_num)]
        available_timeslots = [timeslot for timeslot in timeslots if all(timeslot in rm_available_ts_by_wd[wd] for wd in weekdays_)]

        # find any professors that are available during that timeslot
        for timeslot in available_timeslots:
            for prof_id in prof_options:
                prof_available_ts_by_wd = prof_availabilities[prof_id]
                if all(timeslot in prof_available_ts_by_wd[wd] for wd in weekdays_):
                    # claim resources
                    for weekday in weekdays_:
                        rm_available_ts_by_wd[weekday].remove(timeslot)
                        prof_available_ts_by_wd[weekday].remove(timeslot)
                    return bld_id, rm_num, prof_id, timeslot

In [127]:
np.random.seed(5071)

sect_id = 1
section_rows = []
weekly_session_rows = []

for year in year_to_quarters.keys():
    for quarter in year_to_quarters[year]:
        # room => weekday => open timeslots
        rm_availabilities = {(bld_id,rm_num):{wd:timeslots.copy() for wd in weekdays} for bld_id,rm_num in room[["Bld_id","Rm_num"]].values}

        # professor => weekday => open timeslots
        prof_availabilities = {prof_id:{wd:timeslots.copy() for wd in weekdays} for prof_id in professor["Prof_id"]}

        # schedule 2 sections for each course
        for _,crs in course.sort_values(by="Crs_num").iterrows():
            crs_id = crs["Crs_id"]
            sect_capacity = generate_section_capacity(crs["Crs_num"])

            for section in range(2):
                # generate weekdays
                weekdays_ = [weekdays[i] for i in n_choose_k(len(weekdays),k=3)]

                # schedule section based on sect_capacity and weekdays_
                bld_id, rm_num, prof_id, timeslot = book_section(
                    sect_capacity,
                    crs["Dept_id"],
                    weekdays_,
                    rm_availabilities,
                    prof_availabilities
                )
                section_rows.append({
                    "Sect_id": sect_id,
                    "Crs_id": crs_id,
                    "Prof_id": prof_id,
                    "Sect_capacity": sect_capacity,
                    "Sect_year": year,
                    "Sect_quarter": quarter,
                })
                for weekday in weekdays_:
                    weekly_session_rows.append({
                        "Sect_id": sect_id,
                        "WS_weekday": weekday,
                        "WS_start_time": f"{timeslot}:00",
                        "WS_end_time": f"{timeslot}:50",
                        "Bld_id": bld_id,
                        "Rm_num": rm_num
                    })
                sect_id += 1

In [128]:
section = pd.DataFrame(data=section_rows)
section

Sect_id  Crs_id  Prof_id  Sect_capacity  Sect_year  Sect_quarter
0           1     220      123            110       2023             1
1           2     220       32            110       2023             1
2           3       1      179             85       2023             1
3           4       1      191             85       2023             1
4           5     293        3             90       2023             1
...       ...     ...      ...            ...        ...           ...
7099     7100     118      135             25       2025             4
7100     7101     444      117             30       2025             4
7101     7102     444      168             30       2025             4
7102     7103     279      119             15       2025             4
7103     7104     279       86             15       2025             4

[7104 rows x 6 columns]

In [129]:
weekly_session = pd.DataFrame(data=weekly_session_rows).sort_values(by=["Sect_id","WS_weekday"]).reset_index(drop=True)
weekly_session

Sect_id  WS_weekday WS_start_time WS_end_time  Bld_id Rm_num
0            1           1          8:00        8:50      11    220
1            1           2          8:00        8:50      11    220
2            1           5          8:00        8:50      11    220
3            2           2          8:00        8:50      11    120
4            2           3          8:00        8:50      11    120
...        ...         ...           ...         ...     ...    ...
21307     7103           2         12:00       12:50      12    162
21308     7103           3         12:00       12:50      12    162
21309     7104           1         13:00       13:50       7    134
21310     7104           2         13:00       13:50       7    134
21311     7104           3         13:00       13:50       7    134

[21312 rows x 6 columns]

In [130]:
section.to_csv(out_dir+"/section.csv",header=True,index=False)
weekly_session.to_csv(out_dir+"/weekly_session.csv",header=True,index=False)

## Assignment (tbc)

In [135]:
np.random.seed(5071)

rows = []
for crs_id in course["Crs_id"]:
    # number of assignments that sections of the course will have
    num_assignments = np.random.choice([4,8])  # 4, 8
    for sect_id in section.loc[section["Crs_id"]==crs_id]["Sect_id"]:
        # Create num_assignment homework assignments called "Homework i", uniformly weighted
        for i in range(1,num_assignments+1):
            # create Homework i
            rows.append({"Sect_id":sect_id,"Assn_name":f"Homework {i}","Assn_weight":1/num_assignments})

In [136]:
assignment = pd.DataFrame(data=rows)
insert_id_column(assignment,"Assn_id",0)
assignment

Assn_id  Sect_id   Assn_name  Assn_weight
0            1        3  Homework 1         0.25
1            2        3  Homework 2         0.25
2            3        3  Homework 3         0.25
3            4        3  Homework 4         0.25
4            5        4  Homework 1         0.25
...        ...      ...         ...          ...
43067    43068     7101  Homework 4         0.25
43068    43069     7102  Homework 1         0.25
43069    43070     7102  Homework 2         0.25
43070    43071     7102  Homework 3         0.25
43071    43072     7102  Homework 4         0.25

[43072 rows x 4 columns]

In [137]:
assignment.to_csv(out_dir+"/assignment.csv",header=True,index=False)

## Enrollment, TA, Assignment_Grade, Final_Grade

In [138]:
def normalize(ps):
    s = sum(ps)
    return [p/s for p in ps]

In [139]:
# index results for big for-loop

student_to_major_minors = {stud_id:{"MAJOR":[],"MINOR":[]} for stud_id in student_seed["Stud_id"]}
for _,row in major_minor.iterrows():
    student_to_major_minors[row["Stud_id"]][row["type"]].append(row["Dept_id"])

section_professor = section.merge(professor_seed,on="Prof_id")
section_to_prof_aptitude = {}
for _,row in section_professor.iterrows():
    section_to_prof_aptitude[row["Sect_id"]] = row["temp_aptitude"]

course_to_prereqs = {crs_id:[] for crs_id in course["Crs_id"]}
for _,row in prerequisite.iterrows():
    course_to_prereqs[row["Crs_id"]].append(row["Prereq_crs_id"])

In [140]:
# Given a list of passed courses and section booking information, returns a list of (crs_id,dept_id,sect_id) tuples
# of courses that can be taken (sect_id is an arbitrary open section for the course). Does not include courses already passed.
def get_course_options(courses_passed,open_seats):
    # we can take a course if (1) we have passed all the prerequisites and (2) there is an open section
    options = []
    for crs_id, dept_id in course[["Crs_id","Dept_id"]].values:
        # if (1) we haven't already passed the course and (2) we have passed all the prerequisites, try to find an open section
        if crs_id not in courses_passed and \
           all(prereq_crs_id in courses_passed for prereq_crs_id in course_to_prereqs[crs_id]):
            for sect_id,open_seats_ in open_seats[crs_id].items():
               if open_seats_ > 0:
                   options.append((crs_id,dept_id,sect_id))
                   break
    return options

In [ ]:
# TODO?: student TA positions and enrolled courses aren't guaranteed to have non-conflicting schedules

np.random.seed(5071)

GP_pass_threshold = 1.0

enrollment_rows = []
assignment_grade_rows = []
final_grade_rows = []
TA_rows = []

courses_passed = {stud_id:[] for stud_id in student_seed["Stud_id"]}  # stud_id => list of courses passed
course_grades = {stud_id:[] for stud_id in student_seed["Stud_id"]}  # stud_id => list of grades (between 0 and 1)
for year in year_to_quarters.keys():
    quarters = year_to_quarters[year]
    quarters.sort(key = lambda q: (q-2)%4)  # Wi Y, Sp Y < Su Y < Fa Y
    for quarter in quarters:
        print(year,quarter)
        
        quarter_sections = section.loc[(section["Sect_year"]==year) & (section["Sect_quarter"]==quarter)]

        # crs_id => sect_id => open seats
        quarter_open_seats = {crs_id:{sect_id:section.loc[section["Sect_id"]==sect_id]["Sect_capacity"].iloc[0]
                              for sect_id in quarter_sections.loc[section["Crs_id"]==crs_id]["Sect_id"]}
                              for crs_id in course["Crs_id"]}
        quarter_enrollment = {stud_id:[] for stud_id in student_seed["Stud_id"]}   # stud_id => enrolled (crs_id,sect_id) pairs
        quarter_TA_count = {sect_id:0 for sect_id in quarter_sections["Sect_id"]}  # sect_id => number of TAs

        # register students for courses and TA positions
        for _,student in student_seed.iterrows():
            stud_id = student["Stud_id"]
            courses_passed_ = courses_passed[stud_id]
            course_grades_ = course_grades[stud_id]

            # get the majors and minor of student
            major_dept_ids = student_to_major_minors[stud_id]["MAJOR"]
            minor_dept_ids = student_to_major_minors[stud_id]["MINOR"]

            # if the student has passed some classes, then they are eligible to TA
            if len(courses_passed_) > 0:
                average_course_grade = np.mean(course_grades_)
                p = np.sqrt(restrict(2*(average_course_grade-0.64)+0.04,0.01,1))

                # 100*p% chance to TA a previously passed course
                if np.random.uniform() < p:
                    course_to_TA = np.random.choice(courses_passed_)
                    section_to_TA = np.random.choice([*quarter_open_seats[course_to_TA].keys()])
                    TA_rows.append({"Stud_id":stud_id,"Sect_id":section_to_TA})
                    quarter_TA_count[section_to_TA] += 1

            # get course options and department ids of those courses
            options = get_course_options(courses_passed_,quarter_open_seats)

            # randomly choose 4 courses, weighted towards courses in major/minor
            ps = [200 if dept_id in major_dept_ids else 100 if dept_id in minor_dept_ids else 1 for _,dept_id,_ in options]
            courses_to_take = [options[i] for i in np.random.choice(len(options),p=normalize(ps),size=4,replace=False)]

            # enroll in selected courses
            for crs_id,_,sect_id in courses_to_take:
                enrollment_rows.append({"Stud_id":stud_id,"Sect_id":sect_id})
                quarter_enrollment[stud_id].append((crs_id,sect_id))
                quarter_open_seats[crs_id][sect_id] -= 1
        
        # generate quarter grades
        for _,student in student_seed.iterrows():
            stud_id = student["Stud_id"]
            stud_aptitude = student["temp_aptitude"]
            course_grades_ = course_grades[stud_id]

            for crs_id,sect_id in quarter_enrollment[stud_id]:
                prof_aptitude = section_to_prof_aptitude[sect_id]
                num_TAs = quarter_TA_count[sect_id]

                assignment_grades = []
                for assn_id in assignment.loc[assignment["Sect_id"]==sect_id]["Assn_id"]:
                    grade = float(restrict_01(np.random.normal(0.8+(stud_aptitude-0.5)*0.6+(prof_aptitude-0.5)*0.2+0.03*num_TAs,0.05)))
                    assignment_grades.append(grade)
                    assignment_grade_rows.append({"Assn_id":assn_id,"Stud_id":stud_id,"grade":grade})
                
                # compute overall grade (mean of the grades since they are uniformly weighted) and grade_point
                overall_grade = round(np.mean(assignment_grades),3)
                grade_point = round(restrict_04(4-8*(0.95-overall_grade)),2)
                final_grade_rows.append({"Stud_id":stud_id,"Sect_id":sect_id,"overall_grade":overall_grade,"grade_point":grade_point})
                course_grades_.append(overall_grade)

                # if grade_point is at least 1.0, the student passed the course
                if grade_point >= GP_pass_threshold:
                    courses_passed[stud_id].append(crs_id)

2023 1
2024 2
2024 3
2024 4
2024 1
2025 2
2025 3
2025 4


In [142]:
enrollment = pd.DataFrame(data=enrollment_rows).sort_values(by=["Sect_id","Stud_id"]).reset_index(drop=True)
enrollment

Stud_id  Sect_id
0           18        1
1           30        1
2           74        1
3          108        1
4          134        1
...        ...      ...
95995     1614     7104
95996     1630     7104
95997     1861     7104
95998     1896     7104
95999     1900     7104

[96000 rows x 2 columns]

In [143]:
TA = pd.DataFrame(data=TA_rows).sort_values(by=["Sect_id","Stud_id"]).reset_index(drop=True)
TA

Stud_id  Sect_id
0          236      889
1          931      889
2         2479      889
3         2699      889
4         2917      889
...        ...      ...
14113     2338     7103
14114      162     7104
14115      433     7104
14116      598     7104
14117     1050     7104

[14118 rows x 2 columns]

In [144]:
assignment_grade = pd.DataFrame(data=assignment_grade_rows).sort_values(by=["Assn_id","Stud_id"]).reset_index(drop=True)
assignment_grade

Assn_id  Stud_id     grade
0             1       61  1.000000
1             1       80  0.592411
2             1      161  1.000000
3             1      186  0.846300
4             1      196  1.000000
...         ...      ...       ...
594535    43072     2440  0.818635
594536    43072     2446  0.822020
594537    43072     2453  0.617878
594538    43072     2506  0.941937
594539    43072     2522  0.761360

[594540 rows x 3 columns]

In [145]:
final_grade = pd.DataFrame(data=final_grade_rows).sort_values(by=["Sect_id","Stud_id"]).reset_index(drop=True)
final_grade

Stud_id  Sect_id  overall_grade  grade_point
0           18        1          0.781         2.65
1           30        1          0.632         1.46
2           74        1          0.752         2.42
3          108        1          0.666         1.73
4          134        1          0.550         0.80
...        ...      ...            ...          ...
95995     1614     7104          0.936         3.89
95996     1630     7104          0.864         3.31
95997     1861     7104          0.910         3.68
95998     1896     7104          0.948         3.98
95999     1900     7104          0.988         4.00

[96000 rows x 4 columns]

In [146]:
assignment_grade.to_csv(out_dir+"/assignment_grade.csv",header=True,index=False)
final_grade.to_csv(out_dir+"/final_grade.csv",header=True,index=False)

In [147]:
# drop everything related to empty sections

section["temp_num_enrolled"] = section["Sect_id"].apply(lambda sect_id: len(enrollment.loc[enrollment["Sect_id"]==sect_id]))
empty_sections = section[section["temp_num_enrolled"]==0]["Sect_id"]

section = section[~section["Sect_id"].isin(empty_sections)][["Sect_id","Crs_id","Prof_id","Sect_capacity","Sect_year","Sect_quarter"]]
TA = TA[~TA["Sect_id"].isin(empty_sections)]
assignment = assignment[~assignment["Sect_id"].isin(empty_sections)]
weekly_session = weekly_session[~weekly_session["Sect_id"].isin(empty_sections)]

In [148]:
section.to_csv(out_dir+"/section.csv",header=True,index=False)
weekly_session.to_csv(out_dir+"/weekly_session.csv",header=True,index=False)
assignment.to_csv(out_dir+"/assignment.csv",header=True,index=False)

enrollment.to_csv(out_dir+"/enrollment.csv",header=True,index=False)
TA.to_csv(out_dir+"/TA.csv",header=True,index=False)

## Student (continued)

In [149]:
temp = final_grade.merge(section,on="Sect_id").merge(course,on="Crs_id")
stud_gpa = []
for _,row in student_seed.iterrows():
    temp_stud = temp.loc[temp["Stud_id"]==row["Stud_id"]]
    stud_gpa.append(round(np.sum(temp_stud["Crs_credits"]*temp_stud["grade_point"])/np.sum(temp_stud["Crs_credits"]),2))

student_seed["Stud_GPA"] = stud_gpa
student_seed

Stud_id Stud_fname     Stud_lname              Stud_email  \
0           1   Alasteir     Cattellion    acattellion0@uni.edu   
1           2   Northrup       Haggidon      nhaggidon1@uni.edu   
2           3    Rudiger  Butler-Bowdon  rbutlerbowdon2@uni.edu   
3           4       Mick        Kitcatt       mkitcatt3@uni.edu   
4           5      Boone   Spottiswoode  bspottiswoode4@uni.edu   
...       ...        ...            ...                     ...   
2995     2996     Rickey          Thyer        rthyerrn@uni.edu   
2996     2997     Alejoa           Gane         aganero@uni.edu   
2997     2998      Bessy        Tidbold      btidboldrp@uni.edu   
2998     2999       Esra           Hess         ehessrq@uni.edu   
2999     3000   Christie    Springthorp  cspringthorprr@uni.edu   

      temp_aptitude  Stud_GPA  
0          0.424435      3.34  
1          0.447974      3.44  
2          0.702458      3.91  
3          0.383267      3.24  
4          0.413556      3.18  
...             ...       ...  
2995       0.000000      1.71  
2996       0.177728      2.12  
2997       0.370496      2.88  
2998       0.196102      2.22  
2999       0.348426      3.21  

[3000 rows x 6 columns]

In [150]:
student = student_seed[["Stud_id","Stud_fname","Stud_lname","Stud_email","Stud_GPA"]]
student

Stud_id Stud_fname     Stud_lname              Stud_email  Stud_GPA
0           1   Alasteir     Cattellion    acattellion0@uni.edu      3.34
1           2   Northrup       Haggidon      nhaggidon1@uni.edu      3.44
2           3    Rudiger  Butler-Bowdon  rbutlerbowdon2@uni.edu      3.91
3           4       Mick        Kitcatt       mkitcatt3@uni.edu      3.24
4           5      Boone   Spottiswoode  bspottiswoode4@uni.edu      3.18
...       ...        ...            ...                     ...       ...
2995     2996     Rickey          Thyer        rthyerrn@uni.edu      1.71
2996     2997     Alejoa           Gane         aganero@uni.edu      2.12
2997     2998      Bessy        Tidbold      btidboldrp@uni.edu      2.88
2998     2999       Esra           Hess         ehessrq@uni.edu      2.22
2999     3000   Christie    Springthorp  cspringthorprr@uni.edu      3.21

[3000 rows x 5 columns]

In [151]:
student.to_csv(out_dir+"/student.csv",header=True,index=False)

# Check (Example Usage)

In [162]:
# pick a student
stud_id = 3
student.loc[student["Stud_id"]==stud_id]

Stud_id Stud_fname     Stud_lname              Stud_email  Stud_GPA
2        3    Rudiger  Butler-Bowdon  rbutlerbowdon2@uni.edu      3.91

In [163]:
# major/minors
major_minor.loc[major_minor["Stud_id"]==stud_id].merge(department,on="Dept_id")

Stud_id  Dept_id   type  Col_id                Dept_name Dept_code
0        3        3  MAJOR       1  Department of Economics      ECON
1        3        9  MINOR       2              Real Estate        RE

In [164]:
# transcript
temp = final_grade.loc[final_grade["Stud_id"]==stud_id].merge(section,on=["Sect_id"]).merge(course,on="Crs_id").merge(department,on="Dept_id")
temp["qo"] = (temp["Sect_quarter"]-2)%4
temp = temp.sort_values(by=["Sect_year","qo","Crs_credits"])
temp[["Stud_id","Sect_year","Sect_quarter","Dept_code","Crs_num","Crs_name","Crs_credits","overall_grade","grade_point"]]

Stud_id  Sect_year  Sect_quarter Dept_code  Crs_num  \
0         3       2023             1      ECON      200   
1         3       2023             1      ECON      216   
2         3       2023             1      ECON      235   
3         3       2023             1      ECON      496   
4         3       2024             2      PHYS      106   
5         3       2024             2      ECON      282   
6         3       2024             2      ECON      345   
7         3       2024             2      LING      359   
10        3       2024             3        RE      250   
11        3       2024             3      ECON      346   
8         3       2024             3      ECON      201   
9         3       2024             3      ECON      230   
13        3       2024             4        RE      415   
12        3       2024             4      PHIL      291   
14        3       2024             4      ECON      494   
15        3       2024             4      ECON      497   
17        3       2024             1        RE      418   
16        3       2024             1        RE      413   
19        3       2024             1        RE      464   
18        3       2024             1        RE      462   
21        3       2025             2        RE      416   
20        3       2025             2        RE      397   
22        3       2025             2      PHIL      466   
23        3       2025             2      MATH      499   
25        3       2025             3        RE      361   
26        3       2025             3        RE      411   
27        3       2025             3        RE      497   
24        3       2025             3      MATH      100   
28        3       2025             4        RE      363   
29        3       2025             4        RE      408   
31        3       2025             4        RE      459   
30        3       2025             4        RE      431   

                                             Crs_name  Crs_credits  \
0                      Introduction to Microeconomics            5   
1                                 Economics of Sports            5   
2             Introduction to Environmental Economics            5   
3                                      Honors Seminar            5   
4                       Facilitated Group Inquiry III            2   
5            Using Econometrics: A Practical Approach            5   
6                             Global Health Economics            5   
7                              Language and Ethnicity            5   
10                        Introduction to Real Estate            3   
11  Using Economics to Solve Today's Healthcare Pr...            4   
8                      Introduction to Macroeconomics            5   
9                   Economics of Fisheries and Oceans            5   
13                                    Real Estate Law            3   
12                                  Ethics in Science            5   
14                                   Economy of Japan            5   
15                              Honors Directed Study            5   
17          Best Practices in Sustainable Real Estate            3   
16                 Real Estate Finance and Investment            4   
19                                 Affordable Housing            4   
18             Residential Real Estate Data Analytics            5   
21          Real Estate Economics and Market Analysis            4   
20          Introduction to Real Estate Data Modeling            5   
22                  Philosophy of the Social Sciences            5   
23                             Undergraduate Research            8   
25                              Property Transactions            3   
26                Real Estate Valuation and Appraisal            3   
27                          Real Estate Data Modeling            4   
24                                            Algebra            5   
28                    Real

In [165]:
# TA positions (check that they only TA'd a course after they passed it)
temp = TA.loc[TA["Stud_id"]==stud_id].merge(section,on=["Sect_id"]).merge(course,on="Crs_id").merge(department,on="Dept_id")
temp["qo"] = (temp["Sect_quarter"]-2)%4
temp = temp.sort_values(by=["Sect_year","qo","Crs_credits"])
temp[["Stud_id","Sect_year","Sect_quarter","Dept_code","Crs_num","Crs_name"]]

Stud_id  Sect_year  Sect_quarter Dept_code  Crs_num  \
0        3       2024             3      ECON      216   
1        3       2024             4      ECON      345   
2        3       2024             1      ECON      494   
3        3       2025             3      ECON      345   
4        3       2025             4        RE      415   

                  Crs_name  
0      Economics of Sports  
1  Global Health Economics  
2         Economy of Japan  
3  Global Health Economics  
4          Real Estate Law

In [166]:
# check that taken courses had prerequisites completed
temp_prereq = prerequisite.merge(course,left_on="Prereq_crs_id",right_on="Crs_id").merge(department,on="Dept_id")[["Crs_id_x","Dept_code","Crs_num"]]
temp_prereq = temp_prereq.rename(columns={"Crs_id_x":"Crs_id","Dept_code":"Prereq_dept_code","Crs_num":"Prereq_crs_num"})
temp = final_grade.loc[final_grade["Stud_id"]==stud_id].merge(section,on=["Sect_id"]).merge(course,on="Crs_id").merge(department,on="Dept_id").merge(temp_prereq,on="Crs_id",how="left")
temp["qo"] = (temp["Sect_quarter"]-2)%4
temp = temp.sort_values(by=["Sect_year","qo","Crs_credits"])
temp[["Stud_id","Sect_year","Sect_quarter","Dept_code","Crs_num","Prereq_dept_code","Prereq_crs_num"]]

Stud_id  Sect_year  Sect_quarter Dept_code  Crs_num Prereq_dept_code  \
0         3       2023             1      ECON      200              NaN   
1         3       2023             1      ECON      216              NaN   
2         3       2023             1      ECON      235              NaN   
3         3       2023             1      ECON      496              NaN   
4         3       2024             2      PHYS      106              NaN   
5         3       2024             2      ECON      282              NaN   
6         3       2024             2      ECON      345              NaN   
7         3       2024             2      LING      359              NaN   
10        3       2024             3        RE      250              NaN   
11        3       2024             3      ECON      346             ECON   
8         3       2024             3      ECON      201             ECON   
9         3       2024             3      ECON      230              NaN   
13        3       2024             4        RE      415               RE   
12        3       2024             4      PHIL      291              NaN   
14        3       2024             4      ECON      494             ECON   
15        3       2024             4      ECON      497             ECON   
17        3       2024             1        RE      418               RE   
16        3       2024             1        RE      413               RE   
19        3       2024             1        RE      464               RE   
18        3       2024             1        RE      462               RE   
21        3       2025             2        RE      416               RE   
20        3       2025             2        RE      397               RE   
22        3       2025             2      PHIL      466              NaN   
23        3       2025             2      MATH      499              NaN   
25        3       2025             3        RE      361               RE   
26        3       2025             3        RE      411               RE   
27        3       2025             3        RE      497               RE   
28        3       2025             3        RE      497               RE   
29        3       2025             3        RE      497               RE   
24        3       2025             3      MATH      100              NaN   
30        3       2025             4        RE      363               RE   
31        3       2025             4        RE      408               RE   
34        3       2025             4        RE      459               RE   
32        3       2025             4        RE      431               RE   
33        3       2025             4        RE      431               RE   

    Prereq_crs_num  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN  
5              NaN  
6              NaN  
7              NaN  
10             NaN  
11           200.0  
8            200.0  
9              NaN  
13           250.0  
12             NaN  
14           201.0  
15           496.0  
17           250.0  
16           250.0  
19           250.0  
18           250.0  
21           250.0  
20           250.0  
22             NaN  
23             NaN  
25           250.0  
26           250.0  
27           250.0  
28           397.0  
29           416.0  
24             NaN  
30           250.0  
31           250.0  
34           250.0  
32           250.0  
33           413.0